In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
m_path = os.path.abspath(os.path.join("..\.."))
if m_path not in sys.path:
    sys.path.append(m_path)
    
from models.rigging.weight_transfer import main

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
source_mesh_path = "../no_git_test_data/88-1/smpl1.obj"
target_mesh_path = "../no_git_test_data/88-1/model_cleaned.obj"

main(source_mesh_path, target_mesh_path)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\User\.conda\envs\fyp-xcloth\lib\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_26780\1252107674.py", line 4, in <module>
    main(source_mesh_path, target_mesh_path)
  File "c:\Users\User\CODE\FYP\models\rigging\weight_inpainting.py", line 688, in main
  File "c:\Users\User\CODE\FYP\models\rigging\weight_inpainting.py", line 664, in segregate_vertices_by_confidence
  File "c:\Users\User\CODE\FYP\models\rigging\weight_inpainting.py", line 50, in wrapper
    result = func(*args, **kwargs)
  File "c:\Users\User\CODE\FYP\models\rigging\weight_inpainting.py", line 149, in create_vertex_data_array
    mesh.numVertices,
AttributeError: 'open3d.cpu.pybind.geometry.TriangleMesh' object has no attribute 'numVertices'. Did you mean: 'vertices'?

During handling of the above exception, another exception occurred:

Trac

In [19]:
from smplpytorch.pytorch.smpl_layer import SMPL_Layer

smpl_layer = SMPL_Layer(
    center_idx=0,
    gender='male',
    model_root="C:/Users/User/CODE/FYP/models/smpl",
)

smpl_layer.smpl_data["weights"].shape

(6890, 24)

In [20]:
smpl_layer.smpl_data

{'J_regressor_prior': <24x6890 sparse matrix of type '<class 'numpy.float64'>'
 	with 6810 stored elements in Compressed Sparse Column format>,
 'f': array([[   1,    2,    0],
        [   0,    2,    3],
        [   2,    1,    4],
        ...,
        [4805, 3511, 6309],
        [3511, 1330, 6309],
        [6309, 1330, 4687]], dtype=uint32),
 'J_regressor': <24x6890 sparse matrix of type '<class 'numpy.float64'>'
 	with 226 stored elements in Compressed Sparse Column format>,
 'kintree_table': array([[4294967295,          0,          0,          0,          1,
                  2,          3,          4,          5,          6,
                  7,          8,          9,          9,          9,
                 12,         13,         14,         16,         17,
                 18,         19,         20,         21],
        [         0,          1,          2,          3,          4,
                  5,          6,          7,          8,          9,
                 10,        